In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class GroceryShopSystem:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Pune Super Market - Management System")
        self.root.geometry("1200x800")
        self.root.configure(bg='#f0f0f0')
        
        # Data storage
        self.inventory = {}
        self.sales_history = []
        self.customers = {}
        self.current_cart = {}
        
        # ML Models
        self.sales_predictor = None
        self.recommendation_model = None
        
        # Initialize with sample data
        self.initialize_sample_data()
        
        # Create GUI
        self.create_gui()
        
    def initialize_sample_data(self):
        """Initialize with sample inventory and sales data"""
        sample_inventory = {
            'Rice': {'price': 80, 'stock': 50, 'category': 'Grains'},
            'Wheat': {'price': 60, 'stock': 40, 'category': 'Grains'},
            'Milk': {'price': 45, 'stock': 30, 'category': 'Dairy'},
            'Eggs': {'price': 120, 'stock': 25, 'category': 'Dairy'},
            'Apples': {'price': 150, 'stock': 20, 'category': 'Fruits'},
            'Bananas': {'price': 40, 'stock': 35, 'category': 'Fruits'},
            'Tomatoes': {'price': 30, 'stock': 45, 'category': 'Vegetables'},
            'Onions': {'price': 25, 'stock': 50, 'category': 'Vegetables'},
            'Bread': {'price': 35, 'stock': 15, 'category': 'Bakery'},
            'Oil': {'price': 180, 'stock': 20, 'category': 'Cooking'}
        }
        
        self.inventory = sample_inventory
        
        # Generate sample sales history
        self.generate_sample_sales_history()
        
    def generate_sample_sales_history(self):
        """Generate sample sales data for ML training"""
        products = list(self.inventory.keys())
        
        for i in range(100):  # Generate 100 sample transactions
            date = datetime.now() - timedelta(days=np.random.randint(1, 365))
            customer_id = f"CUST_{np.random.randint(1000, 9999)}"
            
            # Random number of items in transaction
            num_items = np.random.randint(1, 6)
            transaction_items = np.random.choice(products, num_items, replace=False)
            
            for item in transaction_items:
                quantity = np.random.randint(1, 5)
                self.sales_history.append({
                    'date': date,
                    'customer_id': customer_id,
                    'product': item,
                    'quantity': quantity,
                    'price': self.inventory[item]['price'],
                    'total': quantity * self.inventory[item]['price']
                })
    
    def create_gui(self):
        """Create the main GUI"""
        # Main frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Title
        title_label = tk.Label(main_frame, text="🛒 Pune Super Market", 
                              font=('Arial', 24, 'bold'), bg='#f0f0f0', fg='#2c3e50')
        title_label.grid(row=0, column=0, columnspan=3, pady=10)
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.grid(row=1, column=0, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S), pady=10)
        
        # Create tabs
        self.create_inventory_tab()
        self.create_sales_tab()
        self.create_analytics_tab()
        self.create_recommendations_tab()
        
        # Configure grid weights
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(0, weight=1)
        main_frame.rowconfigure(1, weight=1)
        
    def create_inventory_tab(self):
        """Create inventory management tab"""
        inventory_frame = ttk.Frame(self.notebook)
        self.notebook.add(inventory_frame, text="📦 Inventory Management")
        
        # Inventory display
        inventory_label = tk.Label(inventory_frame, text="Current Inventory", 
                                 font=('Arial', 16, 'bold'))
        inventory_label.grid(row=0, column=0, columnspan=3, pady=10)
        
        # Inventory treeview
        columns = ('Product', 'Price (₹)', 'Stock', 'Category')
        self.inventory_tree = ttk.Treeview(inventory_frame, columns=columns, show='headings', height=10)
        
        for col in columns:
            self.inventory_tree.heading(col, text=col)
            self.inventory_tree.column(col, width=150)
        
        self.inventory_tree.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky=(tk.W, tk.E))
        
        # Scrollbar for inventory
        scrollbar = ttk.Scrollbar(inventory_frame, orient=tk.VERTICAL, command=self.inventory_tree.yview)
        scrollbar.grid(row=1, column=3, sticky=(tk.N, tk.S))
        self.inventory_tree.configure(yscrollcommand=scrollbar.set)
        
        # Buttons
        button_frame = ttk.Frame(inventory_frame)
        button_frame.grid(row=2, column=0, columnspan=3, pady=10)
        
        ttk.Button(button_frame, text="Add Product", command=self.add_product).grid(row=0, column=0, padx=5)
        ttk.Button(button_frame, text="Update Stock", command=self.update_stock).grid(row=0, column=1, padx=5)
        ttk.Button(button_frame, text="Remove Product", command=self.remove_product).grid(row=0, column=2, padx=5)
        ttk.Button(button_frame, text="Refresh", command=self.refresh_inventory).grid(row=0, column=3, padx=5)
        
        # Low stock alert
        alert_frame = ttk.Frame(inventory_frame)
        alert_frame.grid(row=3, column=0, columnspan=3, pady=10, sticky=(tk.W, tk.E))
        
        tk.Label(alert_frame, text="⚠️ Low Stock Alerts:", font=('Arial', 12, 'bold')).grid(row=0, column=0, sticky=tk.W)
        
        self.low_stock_text = tk.Text(alert_frame, height=4, width=80)
        self.low_stock_text.grid(row=1, column=0, padx=10, pady=5)
        
        self.refresh_inventory()
        
    def create_sales_tab(self):
        """Create sales/billing tab"""
        sales_frame = ttk.Frame(self.notebook)
        self.notebook.add(sales_frame, text="💰 Sales & Billing")
        
        # Customer info
        customer_frame = ttk.LabelFrame(sales_frame, text="Customer Information", padding="10")
        customer_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(customer_frame, text="Customer ID:").grid(row=0, column=0, sticky=tk.W)
        self.customer_id_var = tk.StringVar()
        ttk.Entry(customer_frame, textvariable=self.customer_id_var, width=20).grid(row=0, column=1, padx=5)
        
        # Product selection
        product_frame = ttk.LabelFrame(sales_frame, text="Product Selection", padding="10")
        product_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(product_frame, text="Product:").grid(row=0, column=0, sticky=tk.W)
        self.product_var = tk.StringVar()
        product_combo = ttk.Combobox(product_frame, textvariable=self.product_var, 
                                   values=list(self.inventory.keys()), width=20)
        product_combo.grid(row=0, column=1, padx=5)
        
        tk.Label(product_frame, text="Quantity:").grid(row=0, column=2, sticky=tk.W, padx=(20,0))
        self.quantity_var = tk.StringVar(value="1")
        ttk.Entry(product_frame, textvariable=self.quantity_var, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Button(product_frame, text="Add to Cart", command=self.add_to_cart).grid(row=0, column=4, padx=10)
        
        # Cart display
        cart_frame = ttk.LabelFrame(sales_frame, text="Shopping Cart", padding="10")
        cart_frame.grid(row=2, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        # Cart treeview
        cart_columns = ('Product', 'Quantity', 'Price', 'Total')
        self.cart_tree = ttk.Treeview(cart_frame, columns=cart_columns, show='headings', height=8)
        
        for col in cart_columns:
            self.cart_tree.heading(col, text=col)
            self.cart_tree.column(col, width=120)
        
        self.cart_tree.grid(row=0, column=0, columnspan=3, padx=10, pady=10, sticky=(tk.W, tk.E))
        
        # Cart buttons
        cart_button_frame = ttk.Frame(cart_frame)
        cart_button_frame.grid(row=1, column=0, columnspan=3, pady=10)
        
        ttk.Button(cart_button_frame, text="Remove Item", command=self.remove_from_cart).grid(row=0, column=0, padx=5)
        ttk.Button(cart_button_frame, text="Clear Cart", command=self.clear_cart).grid(row=0, column=1, padx=5)
        ttk.Button(cart_button_frame, text="Generate Bill", command=self.generate_bill).grid(row=0, column=2, padx=5)
        
        # Total display
        self.total_var = tk.StringVar(value="Total: ₹0.00")
        total_label = tk.Label(cart_frame, textvariable=self.total_var, 
                              font=('Arial', 14, 'bold'), fg='#27ae60')
        total_label.grid(row=2, column=0, columnspan=3, pady=10)
        
    def create_analytics_tab(self):
        """Create analytics tab with ML predictions"""
        analytics_frame = ttk.Frame(self.notebook)
        self.notebook.add(analytics_frame, text="📊 Analytics & Predictions")
        
        # Sales prediction section
        prediction_frame = ttk.LabelFrame(analytics_frame, text="Sales Prediction (ML)", padding="10")
        prediction_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(prediction_frame, text="Product:").grid(row=0, column=0, sticky=tk.W)
        self.predict_product_var = tk.StringVar()
        predict_combo = ttk.Combobox(prediction_frame, textvariable=self.predict_product_var,
                                   values=list(self.inventory.keys()), width=20)
        predict_combo.grid(row=0, column=1, padx=5)
        
        tk.Label(prediction_frame, text="Days ahead:").grid(row=0, column=2, sticky=tk.W, padx=(20,0))
        self.predict_days_var = tk.StringVar(value="7")
        ttk.Entry(prediction_frame, textvariable=self.predict_days_var, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Button(prediction_frame, text="Predict Sales", command=self.predict_sales).grid(row=0, column=4, padx=10)
        
        self.prediction_result = tk.Text(prediction_frame, height=4, width=80)
        self.prediction_result.grid(row=1, column=0, columnspan=5, padx=10, pady=10)
        
        # Train ML model button
        ttk.Button(prediction_frame, text="Train ML Model", command=self.train_ml_model).grid(row=2, column=0, columnspan=5, pady=10)
        
        # Sales statistics
        stats_frame = ttk.LabelFrame(analytics_frame, text="Sales Statistics", padding="10")
        stats_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        ttk.Button(stats_frame, text="Generate Report", command=self.generate_sales_report).grid(row=0, column=0, padx=5)
        ttk.Button(stats_frame, text="Show Top Products", command=self.show_top_products).grid(row=0, column=1, padx=5)
        
        self.stats_text = tk.Text(stats_frame, height=10, width=80)
        self.stats_text.grid(row=1, column=0, columnspan=2, padx=10, pady=10)
        
    def create_recommendations_tab(self):
        """Create recommendations tab"""
        recommendations_frame = ttk.Frame(self.notebook)
        self.notebook.add(recommendations_frame, text="🎯 Recommendations")
        
        # Customer-based recommendations
        customer_rec_frame = ttk.LabelFrame(recommendations_frame, text="Customer Recommendations", padding="10")
        customer_rec_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(customer_rec_frame, text="Customer ID:").grid(row=0, column=0, sticky=tk.W)
        self.rec_customer_var = tk.StringVar()
        ttk.Entry(customer_rec_frame, textvariable=self.rec_customer_var, width=20).grid(row=0, column=1, padx=5)
        
        ttk.Button(customer_rec_frame, text="Get Recommendations", 
                  command=self.get_customer_recommendations).grid(row=0, column=2, padx=10)
        
        self.recommendations_text = tk.Text(customer_rec_frame, height=8, width=80)
        self.recommendations_text.grid(row=1, column=0, columnspan=3, padx=10, pady=10)
        
        # Product associations
        associations_frame = ttk.LabelFrame(recommendations_frame, text="Product Associations", padding="10")
        associations_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        ttk.Button(associations_frame, text="Find Product Associations", 
                  command=self.find_product_associations).grid(row=0, column=0, padx=5)
        
        self.associations_text = tk.Text(associations_frame, height=8, width=80)
        self.associations_text.grid(row=1, column=0, padx=10, pady=10)
    
    def refresh_inventory(self):
        """Refresh inventory display"""
        # Clear existing items
        for item in self.inventory_tree.get_children():
            self.inventory_tree.delete(item)
        
        # Add current inventory
        for product, details in self.inventory.items():
            self.inventory_tree.insert('', 'end', values=(
                product, details['price'], details['stock'], details['category']
            ))
        
        # Update low stock alerts
        self.update_low_stock_alerts()
    
    def update_low_stock_alerts(self):
        """Update low stock alerts"""
        self.low_stock_text.delete(1.0, tk.END)
        low_stock_items = []
        
        for product, details in self.inventory.items():
            if details['stock'] < 10:  # Low stock threshold
                low_stock_items.append(f"{product}: {details['stock']} units left")
        
        if low_stock_items:
            self.low_stock_text.insert(tk.END, "\n".join(low_stock_items))
        else:
            self.low_stock_text.insert(tk.END, "All items are well stocked!")
    
    def add_product(self):
        """Add new product to inventory"""
        product_name = simpledialog.askstring("Add Product", "Enter product name:")
        if not product_name:
            return
        
        price = simpledialog.askfloat("Add Product", "Enter price:")
        if price is None:
            return
            
        stock = simpledialog.askinteger("Add Product", "Enter initial stock:")
        if stock is None:
            return
            
        category = simpledialog.askstring("Add Product", "Enter category:")
        if not category:
            return
        
        self.inventory[product_name] = {
            'price': price,
            'stock': stock,
            'category': category
        }
        
        self.refresh_inventory()
        messagebox.showinfo("Success", f"Product '{product_name}' added successfully!")
    
    def update_stock(self):
        """Update stock for selected product"""
        selected = self.inventory_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select a product to update.")
            return
        
        item = self.inventory_tree.item(selected[0])
        product_name = item['values'][0]
        
        new_stock = simpledialog.askinteger("Update Stock", f"Enter new stock for {product_name}:")
        if new_stock is None:
            return
        
        self.inventory[product_name]['stock'] = new_stock
        self.refresh_inventory()
        messagebox.showinfo("Success", f"Stock updated for '{product_name}'!")
    
    def remove_product(self):
        """Remove selected product from inventory"""
        selected = self.inventory_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select a product to remove.")
            return
        
        item = self.inventory_tree.item(selected[0])
        product_name = item['values'][0]
        
        if messagebox.askyesno("Confirm", f"Are you sure you want to remove '{product_name}'?"):
            del self.inventory[product_name]
            self.refresh_inventory()
            messagebox.showinfo("Success", f"Product '{product_name}' removed successfully!")
    
    def add_to_cart(self):
        """Add selected product to cart"""
        product = self.product_var.get()
        if not product or product not in self.inventory:
            messagebox.showwarning("Warning", "Please select a valid product.")
            return
        
        try:
            quantity = int(self.quantity_var.get())
            if quantity <= 0:
                raise ValueError
        except ValueError:
            messagebox.showwarning("Warning", "Please enter a valid quantity.")
            return
        
        if quantity > self.inventory[product]['stock']:
            messagebox.showwarning("Warning", "Insufficient stock!")
            return
        
        if product in self.current_cart:
            self.current_cart[product] += quantity
        else:
            self.current_cart[product] = quantity
        
        self.update_cart_display()
    
    def update_cart_display(self):
        """Update cart display"""
        # Clear cart tree
        for item in self.cart_tree.get_children():
            self.cart_tree.delete(item)
        
        total = 0
        for product, quantity in self.current_cart.items():
            price = self.inventory[product]['price']
            item_total = price * quantity
            total += item_total
            
            self.cart_tree.insert('', 'end', values=(
                product, quantity, f"₹{price}", f"₹{item_total}"
            ))
        
        self.total_var.set(f"Total: ₹{total:.2f}")
    
    def remove_from_cart(self):
        """Remove selected item from cart"""
        selected = self.cart_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select an item to remove.")
            return
        
        item = self.cart_tree.item(selected[0])
        product_name = item['values'][0]
        
        if product_name in self.current_cart:
            del self.current_cart[product_name]
            self.update_cart_display()
    
    def clear_cart(self):
        """Clear entire cart"""
        self.current_cart.clear()
        self.update_cart_display()
    
    def generate_bill(self):
        """Generate bill and complete transaction"""
        if not self.current_cart:
            messagebox.showwarning("Warning", "Cart is empty!")
            return
        
        customer_id = self.customer_id_var.get()
        if not customer_id:
            customer_id = f"CUST_{np.random.randint(1000, 9999)}"
        
        # Calculate total
        total = 0
        bill_items = []
        
        for product, quantity in self.current_cart.items():
            price = self.inventory[product]['price']
            item_total = price * quantity
            total += item_total
            
            bill_items.append(f"{product} x {quantity} @ ₹{price} = ₹{item_total}")
            
            # Update inventory
            self.inventory[product]['stock'] -= quantity
            
            # Add to sales history
            self.sales_history.append({
                'date': datetime.now(),
                'customer_id': customer_id,
                'product': product,
                'quantity': quantity,
                'price': price,
                'total': item_total
            })
        
        # Generate bill text
        bill_text = f"""
        PUNE SUPER MARKET
        =================
        Customer ID: {customer_id}
        Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        
        Items:
        {chr(10).join(bill_items)}
        
        TOTAL: ₹{total:.2f}
        
        Thank you for shopping with us!
        """
        
        # Show bill
        messagebox.showinfo("Bill Generated", bill_text)
        
        # Clear cart and refresh
        self.clear_cart()
        self.refresh_inventory()
    
    def train_ml_model(self):
        """Train ML model for sales prediction"""
        if not self.sales_history:
            messagebox.showwarning("Warning", "No sales data available for training!")
            return
        
        try:
            # Prepare data
            df = pd.DataFrame(self.sales_history)
            df['date'] = pd.to_datetime(df['date'])
            df['day_of_week'] = df['date'].dt.dayofweek
            df['month'] = df['date'].dt.month
            df['day_of_month'] = df['date'].dt.day
            
            # Group by product and date
            daily_sales = df.groupby(['product', df['date'].dt.date])['quantity'].sum().reset_index()
            
            # Prepare features for each product
            X_list = []
            y_list = []
            
            for product in daily_sales['product'].unique():
                product_data = daily_sales[daily_sales['product'] == product].copy()
                product_data['date'] = pd.to_datetime(product_data['date'])
                product_data = product_data.sort_values('date')
                
                # Create features (day of week, month, day of month, previous sales)
                for i in range(1, len(product_data)):
                    date = product_data.iloc[i]['date']
                    features = [
                        date.dayofweek,
                        date.month,
                        date.day,
                        product_data.iloc[i-1]['quantity']  # Previous day sales
                    ]
                    X_list.append(features)
                    y_list.append(product_data.iloc[i]['quantity'])
            
            if len(X_list) < 10:
                messagebox.showwarning("Warning", "Not enough data for training!")
                return
            
            X = np.array(X_list)
            y = np.array(y_list)
            
            # Train model
            self.sales_predictor = RandomForestRegressor(n_estimators=100, random_state=42)
            self.sales_predictor.fit(X, y)
            
            messagebox.showinfo("Success", "ML Model trained successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error training model: {str(e)}")
    
    def predict_sales(self):
        """Predict sales for selected product"""
        if not self.sales_predictor:
            messagebox.showwarning("Warning", "Please train the ML model first!")
            return
        
        product = self.predict_product_var.get()
        if not product:
            messagebox.showwarning("Warning", "Please select a product.")
            return
        
        try:
            days_ahead = int(self.predict_days_var.get())
            
            # Get recent sales for the product
            recent_sales = [sale for sale in self.sales_history 
                          if sale['product'] == product]
            
            if not recent_sales:
                self.prediction_result.delete(1.0, tk.END)
                self.prediction_result.insert(tk.END, f"No sales history found for {product}")
                return
            
            # Get last sale quantity
            last_quantity = recent_sales[-1]['quantity']
            
            predictions = []
            for i in range(days_ahead):
                future_date = datetime.now() + timedelta(days=i+1)
                features = [
                    future_date.dayofweek,
                    future_date.month,
                    future_date.day,
                    last_quantity
                ]
                
                prediction = self.sales_predictor.predict([features])[0]
                predictions.append(max(0, int(prediction)))  # Ensure non-negative
                last_quantity = prediction
            
            # Display predictions
            self.prediction_result.delete(1.0, tk.END)
            result_text = f"Sales Prediction for {product}:\n\n"
            
            for i, pred in enumerate(predictions):
                future_date = datetime.now() + timedelta(days=i+1)
                result_text += f"Day {i+1} ({future_date.strftime('%Y-%m-%d')}): {pred} units\n"
            
            total_predicted = sum(predictions)
            result_text += f"\nTotal predicted sales for {days_ahead} days: {total_predicted} units"
            
            self.prediction_result.insert(tk.END, result_text)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error making prediction: {str(e)}")
    
    def generate_sales_report(self):
        """Generate comprehensive sales report"""
        if not self.sales_history:
            self.stats_text.delete(1.0, tk.END)
            self.stats_text.insert(tk.END, "No sales data available!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Calculate statistics
        total_sales = df['total'].sum()
        total_transactions = len(df)
        avg_transaction = total_sales / total_transactions if total_transactions > 0 else 0
        
        # Top products by quantity
        top_products_qty = df.groupby('product')['quantity'].sum().sort_values(ascending=False).head(5)
        
        # Top products by revenue
        top_products_revenue = df.groupby('product')['total'].sum().sort_values(ascending=False).head(5)
        
        # Daily sales trend (last 7 days)
        df['date'] = pd.to_datetime(df['date'])
        recent_sales = df[df['date'] >= datetime.now() - timedelta(days=7)]
        daily_sales = recent_sales.groupby(recent_sales['date'].dt.date)['total'].sum()
        
        # Generate report
        report = f"""
SALES REPORT - PUNE SUPER MARKET
================================

OVERVIEW:
- Total Revenue: ₹{total_sales:.2f}
- Total Transactions: {total_transactions}
- Average Transaction Value: ₹{avg_transaction:.2f}

TOP PRODUCTS BY QUANTITY:
{chr(10).join([f"- {product}: {qty} units" for product, qty in top_products_qty.items()])}

TOP PRODUCTS BY REVENUE:
{chr(10).join([f"- {product}: ₹{revenue:.2f}" for product, revenue in top_products_revenue.items()])}

DAILY SALES TREND (Last 7 Days):
{chr(10).join([f"- {date}: ₹{sales:.2f}" for date, sales in daily_sales.items()])}
        """
        
        self.stats_text.delete(1.0, tk.END)
        self.stats_text.insert(tk.END, report)
    
    def show_top_products(self):
        """Show top-selling products"""
        if not self.sales_history:
            self.stats_text.delete(1.0, tk.END)
            self.stats_text.insert(tk.END, "No sales data available!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Calculate product rankings
        product_stats = df.groupby('product').agg({

SyntaxError: incomplete input (4261741318.py, line 652)

In [2]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class GroceryShopSystem:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Pune Super Market - Management System")
        self.root.geometry("1200x800")
        self.root.configure(bg='#f0f0f0')
        
        # Data storage
        self.inventory = {}
        self.sales_history = []
        self.customers = {}
        self.current_cart = {}
        
        # ML Models
        self.sales_predictor = None
        self.recommendation_model = None
        
        # Initialize with sample data
        self.initialize_sample_data()
        
        # Create GUI
        self.create_gui()
        
    def initialize_sample_data(self):
        """Initialize with sample inventory and sales data"""
        sample_inventory = {
            'Rice': {'price': 80, 'stock': 50, 'category': 'Grains'},
            'Wheat': {'price': 60, 'stock': 40, 'category': 'Grains'},
            'Milk': {'price': 45, 'stock': 30, 'category': 'Dairy'},
            'Eggs': {'price': 120, 'stock': 25, 'category': 'Dairy'},
            'Apples': {'price': 150, 'stock': 20, 'category': 'Fruits'},
            'Bananas': {'price': 40, 'stock': 35, 'category': 'Fruits'},
            'Tomatoes': {'price': 30, 'stock': 45, 'category': 'Vegetables'},
            'Onions': {'price': 25, 'stock': 50, 'category': 'Vegetables'},
            'Bread': {'price': 35, 'stock': 15, 'category': 'Bakery'},
            'Oil': {'price': 180, 'stock': 20, 'category': 'Cooking'}
        }
        
        self.inventory = sample_inventory
        
        # Generate sample sales history
        self.generate_sample_sales_history()
        
    def generate_sample_sales_history(self):
        """Generate sample sales data for ML training"""
        products = list(self.inventory.keys())
        
        for i in range(100):  # Generate 100 sample transactions
            date = datetime.now() - timedelta(days=np.random.randint(1, 365))
            customer_id = f"CUST_{np.random.randint(1000, 9999)}"
            
            # Random number of items in transaction
            num_items = np.random.randint(1, 6)
            transaction_items = np.random.choice(products, num_items, replace=False)
            
            for item in transaction_items:
                quantity = np.random.randint(1, 5)
                self.sales_history.append({
                    'date': date,
                    'customer_id': customer_id,
                    'product': item,
                    'quantity': quantity,
                    'price': self.inventory[item]['price'],
                    'total': quantity * self.inventory[item]['price']
                })
    
    def create_gui(self):
        """Create the main GUI"""
        # Main frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Title
        title_label = tk.Label(main_frame, text="🛒 Pune Super Market", 
                              font=('Arial', 24, 'bold'), bg='#f0f0f0', fg='#2c3e50')
        title_label.grid(row=0, column=0, columnspan=3, pady=10)
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.grid(row=1, column=0, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S), pady=10)
        
        # Create tabs
        self.create_inventory_tab()
        self.create_sales_tab()
        self.create_analytics_tab()
        self.create_recommendations_tab()
        
        # Configure grid weights
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(0, weight=1)
        main_frame.rowconfigure(1, weight=1)
        
    def create_inventory_tab(self):
        """Create inventory management tab"""
        inventory_frame = ttk.Frame(self.notebook)
        self.notebook.add(inventory_frame, text="📦 Inventory Management")
        
        # Inventory display
        inventory_label = tk.Label(inventory_frame, text="Current Inventory", 
                                 font=('Arial', 16, 'bold'))
        inventory_label.grid(row=0, column=0, columnspan=3, pady=10)
        
        # Inventory treeview
        columns = ('Product', 'Price (₹)', 'Stock', 'Category')
        self.inventory_tree = ttk.Treeview(inventory_frame, columns=columns, show='headings', height=10)
        
        for col in columns:
            self.inventory_tree.heading(col, text=col)
            self.inventory_tree.column(col, width=150)
        
        self.inventory_tree.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky=(tk.W, tk.E))
        
        # Scrollbar for inventory
        scrollbar = ttk.Scrollbar(inventory_frame, orient=tk.VERTICAL, command=self.inventory_tree.yview)
        scrollbar.grid(row=1, column=3, sticky=(tk.N, tk.S))
        self.inventory_tree.configure(yscrollcommand=scrollbar.set)
        
        # Buttons
        button_frame = ttk.Frame(inventory_frame)
        button_frame.grid(row=2, column=0, columnspan=3, pady=10)
        
        ttk.Button(button_frame, text="Add Product", command=self.add_product).grid(row=0, column=0, padx=5)
        ttk.Button(button_frame, text="Update Stock", command=self.update_stock).grid(row=0, column=1, padx=5)
        ttk.Button(button_frame, text="Remove Product", command=self.remove_product).grid(row=0, column=2, padx=5)
        ttk.Button(button_frame, text="Refresh", command=self.refresh_inventory).grid(row=0, column=3, padx=5)
        
        # Low stock alert
        alert_frame = ttk.Frame(inventory_frame)
        alert_frame.grid(row=3, column=0, columnspan=3, pady=10, sticky=(tk.W, tk.E))
        
        tk.Label(alert_frame, text="⚠️ Low Stock Alerts:", font=('Arial', 12, 'bold')).grid(row=0, column=0, sticky=tk.W)
        
        self.low_stock_text = tk.Text(alert_frame, height=4, width=80)
        self.low_stock_text.grid(row=1, column=0, padx=10, pady=5)
        
        self.refresh_inventory()
        
    def create_sales_tab(self):
        """Create sales/billing tab"""
        sales_frame = ttk.Frame(self.notebook)
        self.notebook.add(sales_frame, text="💰 Sales & Billing")
        
        # Customer info
        customer_frame = ttk.LabelFrame(sales_frame, text="Customer Information", padding="10")
        customer_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(customer_frame, text="Customer ID:").grid(row=0, column=0, sticky=tk.W)
        self.customer_id_var = tk.StringVar()
        ttk.Entry(customer_frame, textvariable=self.customer_id_var, width=20).grid(row=0, column=1, padx=5)
        
        # Product selection
        product_frame = ttk.LabelFrame(sales_frame, text="Product Selection", padding="10")
        product_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(product_frame, text="Product:").grid(row=0, column=0, sticky=tk.W)
        self.product_var = tk.StringVar()
        product_combo = ttk.Combobox(product_frame, textvariable=self.product_var, 
                                   values=list(self.inventory.keys()), width=20)
        product_combo.grid(row=0, column=1, padx=5)
        
        tk.Label(product_frame, text="Quantity:").grid(row=0, column=2, sticky=tk.W, padx=(20,0))
        self.quantity_var = tk.StringVar(value="1")
        ttk.Entry(product_frame, textvariable=self.quantity_var, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Button(product_frame, text="Add to Cart", command=self.add_to_cart).grid(row=0, column=4, padx=10)
        
        # Cart display
        cart_frame = ttk.LabelFrame(sales_frame, text="Shopping Cart", padding="10")
        cart_frame.grid(row=2, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        # Cart treeview
        cart_columns = ('Product', 'Quantity', 'Price', 'Total')
        self.cart_tree = ttk.Treeview(cart_frame, columns=cart_columns, show='headings', height=8)
        
        for col in cart_columns:
            self.cart_tree.heading(col, text=col)
            self.cart_tree.column(col, width=120)
        
        self.cart_tree.grid(row=0, column=0, columnspan=3, padx=10, pady=10, sticky=(tk.W, tk.E))
        
        # Cart buttons
        cart_button_frame = ttk.Frame(cart_frame)
        cart_button_frame.grid(row=1, column=0, columnspan=3, pady=10)
        
        ttk.Button(cart_button_frame, text="Remove Item", command=self.remove_from_cart).grid(row=0, column=0, padx=5)
        ttk.Button(cart_button_frame, text="Clear Cart", command=self.clear_cart).grid(row=0, column=1, padx=5)
        ttk.Button(cart_button_frame, text="Generate Bill", command=self.generate_bill).grid(row=0, column=2, padx=5)
        
        # Total display
        self.total_var = tk.StringVar(value="Total: ₹0.00")
        total_label = tk.Label(cart_frame, textvariable=self.total_var, 
                              font=('Arial', 14, 'bold'), fg='#27ae60')
        total_label.grid(row=2, column=0, columnspan=3, pady=10)
        
    def create_analytics_tab(self):
        """Create analytics tab with ML predictions"""
        analytics_frame = ttk.Frame(self.notebook)
        self.notebook.add(analytics_frame, text="📊 Analytics & Predictions")
        
        # Sales prediction section
        prediction_frame = ttk.LabelFrame(analytics_frame, text="Sales Prediction (ML)", padding="10")
        prediction_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(prediction_frame, text="Product:").grid(row=0, column=0, sticky=tk.W)
        self.predict_product_var = tk.StringVar()
        predict_combo = ttk.Combobox(prediction_frame, textvariable=self.predict_product_var,
                                   values=list(self.inventory.keys()), width=20)
        predict_combo.grid(row=0, column=1, padx=5)
        
        tk.Label(prediction_frame, text="Days ahead:").grid(row=0, column=2, sticky=tk.W, padx=(20,0))
        self.predict_days_var = tk.StringVar(value="7")
        ttk.Entry(prediction_frame, textvariable=self.predict_days_var, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Button(prediction_frame, text="Predict Sales", command=self.predict_sales).grid(row=0, column=4, padx=10)
        
        self.prediction_result = tk.Text(prediction_frame, height=4, width=80)
        self.prediction_result.grid(row=1, column=0, columnspan=5, padx=10, pady=10)
        
        # Train ML model button
        ttk.Button(prediction_frame, text="Train ML Model", command=self.train_ml_model).grid(row=2, column=0, columnspan=5, pady=10)
        
        # Sales statistics
        stats_frame = ttk.LabelFrame(analytics_frame, text="Sales Statistics", padding="10")
        stats_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        ttk.Button(stats_frame, text="Generate Report", command=self.generate_sales_report).grid(row=0, column=0, padx=5)
        ttk.Button(stats_frame, text="Show Top Products", command=self.show_top_products).grid(row=0, column=1, padx=5)
        
        self.stats_text = tk.Text(stats_frame, height=10, width=80)
        self.stats_text.grid(row=1, column=0, columnspan=2, padx=10, pady=10)
        
    def create_recommendations_tab(self):
        """Create recommendations tab"""
        recommendations_frame = ttk.Frame(self.notebook)
        self.notebook.add(recommendations_frame, text="🎯 Recommendations")
        
        # Customer-based recommendations
        customer_rec_frame = ttk.LabelFrame(recommendations_frame, text="Customer Recommendations", padding="10")
        customer_rec_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        tk.Label(customer_rec_frame, text="Customer ID:").grid(row=0, column=0, sticky=tk.W)
        self.rec_customer_var = tk.StringVar()
        ttk.Entry(customer_rec_frame, textvariable=self.rec_customer_var, width=20).grid(row=0, column=1, padx=5)
        
        ttk.Button(customer_rec_frame, text="Get Recommendations", 
                  command=self.get_customer_recommendations).grid(row=0, column=2, padx=10)
        
        self.recommendations_text = tk.Text(customer_rec_frame, height=8, width=80)
        self.recommendations_text.grid(row=1, column=0, columnspan=3, padx=10, pady=10)
        
        # Product associations
        associations_frame = ttk.LabelFrame(recommendations_frame, text="Product Associations", padding="10")
        associations_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), padx=10, pady=5)
        
        ttk.Button(associations_frame, text="Find Product Associations", 
                  command=self.find_product_associations).grid(row=0, column=0, padx=5)
        
        self.associations_text = tk.Text(associations_frame, height=8, width=80)
        self.associations_text.grid(row=1, column=0, padx=10, pady=10)
    
    def refresh_inventory(self):
        """Refresh inventory display"""
        # Clear existing items
        for item in self.inventory_tree.get_children():
            self.inventory_tree.delete(item)
        
        # Add current inventory
        for product, details in self.inventory.items():
            self.inventory_tree.insert('', 'end', values=(
                product, details['price'], details['stock'], details['category']
            ))
        
        # Update low stock alerts
        self.update_low_stock_alerts()
    
    def update_low_stock_alerts(self):
        """Update low stock alerts"""
        self.low_stock_text.delete(1.0, tk.END)
        low_stock_items = []
        
        for product, details in self.inventory.items():
            if details['stock'] < 10:  # Low stock threshold
                low_stock_items.append(f"{product}: {details['stock']} units left")
        
        if low_stock_items:
            self.low_stock_text.insert(tk.END, "\n".join(low_stock_items))
        else:
            self.low_stock_text.insert(tk.END, "All items are well stocked!")
    
    def add_product(self):
        """Add new product to inventory"""
        product_name = simpledialog.askstring("Add Product", "Enter product name:")
        if not product_name:
            return
        
        price = simpledialog.askfloat("Add Product", "Enter price:")
        if price is None:
            return
            
        stock = simpledialog.askinteger("Add Product", "Enter initial stock:")
        if stock is None:
            return
            
        category = simpledialog.askstring("Add Product", "Enter category:")
        if not category:
            return
        
        self.inventory[product_name] = {
            'price': price,
            'stock': stock,
            'category': category
        }
        
        self.refresh_inventory()
        messagebox.showinfo("Success", f"Product '{product_name}' added successfully!")
    
    def update_stock(self):
        """Update stock for selected product"""
        selected = self.inventory_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select a product to update.")
            return
        
        item = self.inventory_tree.item(selected[0])
        product_name = item['values'][0]
        
        new_stock = simpledialog.askinteger("Update Stock", f"Enter new stock for {product_name}:")
        if new_stock is None:
            return
        
        self.inventory[product_name]['stock'] = new_stock
        self.refresh_inventory()
        messagebox.showinfo("Success", f"Stock updated for '{product_name}'!")
    
    def remove_product(self):
        """Remove selected product from inventory"""
        selected = self.inventory_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select a product to remove.")
            return
        
        item = self.inventory_tree.item(selected[0])
        product_name = item['values'][0]
        
        if messagebox.askyesno("Confirm", f"Are you sure you want to remove '{product_name}'?"):
            del self.inventory[product_name]
            self.refresh_inventory()
            messagebox.showinfo("Success", f"Product '{product_name}' removed successfully!")
    
    def add_to_cart(self):
        """Add selected product to cart"""
        product = self.product_var.get()
        if not product or product not in self.inventory:
            messagebox.showwarning("Warning", "Please select a valid product.")
            return
        
        try:
            quantity = int(self.quantity_var.get())
            if quantity <= 0:
                raise ValueError
        except ValueError:
            messagebox.showwarning("Warning", "Please enter a valid quantity.")
            return
        
        if quantity > self.inventory[product]['stock']:
            messagebox.showwarning("Warning", "Insufficient stock!")
            return
        
        if product in self.current_cart:
            self.current_cart[product] += quantity
        else:
            self.current_cart[product] = quantity
        
        self.update_cart_display()
    
    def update_cart_display(self):
        """Update cart display"""
        # Clear cart tree
        for item in self.cart_tree.get_children():
            self.cart_tree.delete(item)
        
        total = 0
        for product, quantity in self.current_cart.items():
            price = self.inventory[product]['price']
            item_total = price * quantity
            total += item_total
            
            self.cart_tree.insert('', 'end', values=(
                product, quantity, f"₹{price}", f"₹{item_total}"
            ))
        
        self.total_var.set(f"Total: ₹{total:.2f}")
    
    def remove_from_cart(self):
        """Remove selected item from cart"""
        selected = self.cart_tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Please select an item to remove.")
            return
        
        item = self.cart_tree.item(selected[0])
        product_name = item['values'][0]
        
        if product_name in self.current_cart:
            del self.current_cart[product_name]
            self.update_cart_display()
    
    def clear_cart(self):
        """Clear entire cart"""
        self.current_cart.clear()
        self.update_cart_display()
    
    def generate_bill(self):
        """Generate bill and complete transaction"""
        if not self.current_cart:
            messagebox.showwarning("Warning", "Cart is empty!")
            return
        
        customer_id = self.customer_id_var.get()
        if not customer_id:
            customer_id = f"CUST_{np.random.randint(1000, 9999)}"
        
        # Calculate total
        total = 0
        bill_items = []
        
        for product, quantity in self.current_cart.items():
            price = self.inventory[product]['price']
            item_total = price * quantity
            total += item_total
            
            bill_items.append(f"{product} x {quantity} @ ₹{price} = ₹{item_total}")
            
            # Update inventory
            self.inventory[product]['stock'] -= quantity
            
            # Add to sales history
            self.sales_history.append({
                'date': datetime.now(),
                'customer_id': customer_id,
                'product': product,
                'quantity': quantity,
                'price': price,
                'total': item_total
            })
        
        # Generate bill text
        bill_text = f"""
        PUNE SUPER MARKET
        =================
        Customer ID: {customer_id}
        Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        
        Items:
        {chr(10).join(bill_items)}
        
        TOTAL: ₹{total:.2f}
        
        Thank you for shopping with us!
        """
        
        # Show bill
        messagebox.showinfo("Bill Generated", bill_text)
        
        # Clear cart and refresh
        self.clear_cart()
        self.refresh_inventory()
    
    def train_ml_model(self):
        """Train ML model for sales prediction"""
        if not self.sales_history:
            messagebox.showwarning("Warning", "No sales data available for training!")
            return
        
        try:
            # Prepare data
            df = pd.DataFrame(self.sales_history)
            df['date'] = pd.to_datetime(df['date'])
            df['day_of_week'] = df['date'].dt.dayofweek
            df['month'] = df['date'].dt.month
            df['day_of_month'] = df['date'].dt.day
            
            # Group by product and date
            daily_sales = df.groupby(['product', df['date'].dt.date])['quantity'].sum().reset_index()
            
            # Prepare features for each product
            X_list = []
            y_list = []
            
            for product in daily_sales['product'].unique():
                product_data = daily_sales[daily_sales['product'] == product].copy()
                product_data['date'] = pd.to_datetime(product_data['date'])
                product_data = product_data.sort_values('date')
                
                # Create features (day of week, month, day of month, previous sales)
                for i in range(1, len(product_data)):
                    date = product_data.iloc[i]['date']
                    features = [
                        date.dayofweek,
                        date.month,
                        date.day,
                        product_data.iloc[i-1]['quantity']  # Previous day sales
                    ]
                    X_list.append(features)
                    y_list.append(product_data.iloc[i]['quantity'])
            
            if len(X_list) < 10:
                messagebox.showwarning("Warning", "Not enough data for training!")
                return
            
            X = np.array(X_list)
            y = np.array(y_list)
            
            # Train model
            self.sales_predictor = RandomForestRegressor(n_estimators=100, random_state=42)
            self.sales_predictor.fit(X, y)
            
            messagebox.showinfo("Success", "ML Model trained successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error training model: {str(e)}")
    
    def predict_sales(self):
        """Predict sales for selected product"""
        if not self.sales_predictor:
            messagebox.showwarning("Warning", "Please train the ML model first!")
            return
        
        product = self.predict_product_var.get()
        if not product:
            messagebox.showwarning("Warning", "Please select a product.")
            return
        
        try:
            days_ahead = int(self.predict_days_var.get())
            
            # Get recent sales for the product
            recent_sales = [sale for sale in self.sales_history 
                          if sale['product'] == product]
            
            if not recent_sales:
                self.prediction_result.delete(1.0, tk.END)
                self.prediction_result.insert(tk.END, f"No sales history found for {product}")
                return
            
            # Get last sale quantity
            last_quantity = recent_sales[-1]['quantity']
            
            predictions = []
            for i in range(days_ahead):
                future_date = datetime.now() + timedelta(days=i+1)
                features = [
                    future_date.dayofweek,
                    future_date.month,
                    future_date.day,
                    last_quantity
                ]
                
                prediction = self.sales_predictor.predict([features])[0]
                predictions.append(max(0, int(prediction)))  # Ensure non-negative
                last_quantity = prediction
            
            # Display predictions
            self.prediction_result.delete(1.0, tk.END)
            result_text = f"Sales Prediction for {product}:\n\n"
            
            for i, pred in enumerate(predictions):
                future_date = datetime.now() + timedelta(days=i+1)
                result_text += f"Day {i+1} ({future_date.strftime('%Y-%m-%d')}): {pred} units\n"
            
            total_predicted = sum(predictions)
            result_text += f"\nTotal predicted sales for {days_ahead} days: {total_predicted} units"
            
            self.prediction_result.insert(tk.END, result_text)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error making prediction: {str(e)}")
    
    def generate_sales_report(self):
        """Generate comprehensive sales report"""
        if not self.sales_history:
            self.stats_text.delete(1.0, tk.END)
            self.stats_text.insert(tk.END, "No sales data available!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Calculate statistics
        total_sales = df['total'].sum()
        total_transactions = len(df)
        avg_transaction = total_sales / total_transactions if total_transactions > 0 else 0
        
        # Top products by quantity
        top_products_qty = df.groupby('product')['quantity'].sum().sort_values(ascending=False).head(5)
        
        # Top products by revenue
        top_products_revenue = df.groupby('product')['total'].sum().sort_values(ascending=False).head(5)
        
        # Daily sales trend (last 7 days)
        df['date'] = pd.to_datetime(df['date'])
        recent_sales = df[df['date'] >= datetime.now() - timedelta(days=7)]
        daily_sales = recent_sales.groupby(recent_sales['date'].dt.date)['total'].sum()
        
        # Generate report
        report = f"""
SALES REPORT - PUNE SUPER MARKET
================================

OVERVIEW:
- Total Revenue: ₹{total_sales:.2f}
- Total Transactions: {total_transactions}
- Average Transaction Value: ₹{avg_transaction:.2f}

TOP PRODUCTS BY QUANTITY:
{chr(10).join([f"- {product}: {qty} units" for product, qty in top_products_qty.items()])}

TOP PRODUCTS BY REVENUE:
{chr(10).join([f"- {product}: ₹{revenue:.2f}" for product, revenue in top_products_revenue.items()])}

DAILY SALES TREND (Last 7 Days):
{chr(10).join([f"- {date}: ₹{sales:.2f}" for date, sales in daily_sales.items()])}
        """
        
        self.stats_text.delete(1.0, tk.END)
        self.stats_text.insert(tk.END, report)
    
    def show_top_products(self):
        """Show top-selling products"""
        if not self.sales_history:
            self.stats_text.delete(1.0, tk.END)
            self.stats_text.insert(tk.END, "No sales data available!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Calculate product rankings
        product_stats = df.groupby('product').agg({
            'quantity': 'sum',
            'total': 'sum',
            'date': 'count'
        }).rename(columns={'date': 'transactions'})
        
        product_stats['avg_price'] = product_stats['total'] / product_stats['quantity']
        product_stats = product_stats.sort_values('total', ascending=False)
        
        report = "TOP PERFORMING PRODUCTS:\n"
        report += "=" * 50 + "\n\n"
        
        for i, (product, stats) in enumerate(product_stats.head(10).iterrows(), 1):
            report += f"{i}. {product}\n"
            report += f"   - Total Revenue: ₹{stats['total']:.2f}\n"
            report += f"   - Units Sold: {stats['quantity']}\n"
            report += f"   - Transactions: {stats['transactions']}\n"
            report += f"   - Avg Price: ₹{stats['avg_price']:.2f}\n\n"
        
        self.stats_text.delete(1.0, tk.END)
        self.stats_text.insert(tk.END, report)
    
    def get_customer_recommendations(self):
        """Get product recommendations for a customer"""
        customer_id = self.rec_customer_var.get()
        if not customer_id:
            messagebox.showwarning("Warning", "Please enter a customer ID.")
            return
        
        if not self.sales_history:
            self.recommendations_text.delete(1.0, tk.END)
            self.recommendations_text.insert(tk.END, "No sales data available for recommendations!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Get customer's purchase history
        customer_purchases = df[df['customer_id'] == customer_id]['product'].unique()
        
        if len(customer_purchases) == 0:
            self.recommendations_text.delete(1.0, tk.END)
            self.recommendations_text.insert(tk.END, f"No purchase history found for customer {customer_id}")
            return
        
        # Find similar customers based on purchase patterns
        customer_product_matrix = df.pivot_table(
            index='customer_id', 
            columns='product', 
            values='quantity', 
            fill_value=0
        )
        
        if customer_id not in customer_product_matrix.index:
            self.recommendations_text.delete(1.0, tk.END)
            self.recommendations_text.insert(tk.END, f"Customer {customer_id} not found in purchase matrix")
            return
        
        # Calculate similarity with all customers
        customer_vector = customer_product_matrix.loc[customer_id].values.reshape(1, -1)
        similarities = cosine_similarity(customer_vector, customer_product_matrix.values)[0]
        
        # Find most similar customers
        similar_customers_idx = np.argsort(similarities)[::-1][1:6]  # Top 5 similar (excluding self)
        similar_customers = customer_product_matrix.index[similar_customers_idx]
        
        # Get products bought by similar customers but not by target customer
        recommended_products = set()
        for similar_customer in similar_customers:
            similar_purchases = customer_product_matrix.loc[similar_customer]
            for product, quantity in similar_purchases.items():
                if quantity > 0 and product not in customer_purchases:
                    recommended_products.add(product)
        
        # Display recommendations
        self.recommendations_text.delete(1.0, tk.END)
        
        if recommended_products:
            recommendation_text = f"RECOMMENDATIONS FOR CUSTOMER {customer_id}:\n"
            recommendation_text += "=" * 50 + "\n\n"
            recommendation_text += f"Based on your purchase history: {', '.join(customer_purchases)}\n\n"
            recommendation_text += "We recommend:\n"
            
            for i, product in enumerate(list(recommended_products)[:5], 1):
                if product in self.inventory:
                    price = self.inventory[product]['price']
                    stock = self.inventory[product]['stock']
                    category = self.inventory[product]['category']
                    recommendation_text += f"{i}. {product}\n"
                    recommendation_text += f"   - Price: ₹{price}\n"
                    recommendation_text += f"   - Category: {category}\n"
                    recommendation_text += f"   - In Stock: {stock} units\n\n"
            
            self.recommendations_text.insert(tk.END, recommendation_text)
        else:
            self.recommendations_text.insert(tk.END, f"No new recommendations found for customer {customer_id}")
    
    def find_product_associations(self):
        """Find products that are frequently bought together"""
        if not self.sales_history:
            self.associations_text.delete(1.0, tk.END)
            self.associations_text.insert(tk.END, "No sales data available!")
            return
        
        df = pd.DataFrame(self.sales_history)
        
        # Group by customer and date to find products bought together
        transactions = df.groupby(['customer_id', df['date'].dt.date])['product'].apply(list).reset_index()
        
        # Find product pairs
        product_pairs = defaultdict(int)
        
        for _, row in transactions.iterrows():
            products = row['product']
            if len(products) > 1:
                for i in range(len(products)):
                    for j in range(i+1, len(products)):
                        pair = tuple(sorted([products[i], products[j]]))
                        product_pairs[pair] += 1
        
        # Sort by frequency
        sorted_pairs = sorted(product_pairs.items(), key=lambda x: x[1], reverse=True)
        
        # Display associations
        self.associations_text.delete(1.0, tk.END)
        
        if sorted_pairs:
            association_text = "PRODUCT ASSOCIATIONS:\n"
            association_text += "=" * 40 + "\n\n"
            association_text += "Products frequently bought together:\n\n"
            
            for i, ((product1, product2), count) in enumerate(sorted_pairs[:10], 1):
                association_text += f"{i}. {product1} + {product2}\n"
                association_text += f"   Bought together {count} times\n\n"
            
            self.associations_text.insert(tk.END, association_text)
        else:
            self.associations_text.insert(tk.END, "No product associations found!")
    
    def run(self):
        """Start the application"""
        self.root.mainloop()

# Additional utility functions for the system
class InventoryOptimizer:
    """Class for inventory optimization using ML"""
    
    def __init__(self, inventory, sales_history):
        self.inventory = inventory
        self.sales_history = sales_history
    
    def optimize_stock_levels(self):
        """Optimize stock levels based on sales patterns"""
        if not self.sales_history:
            return {}
        
        df = pd.DataFrame(self.sales_history)
        df['date'] = pd.to_datetime(df['date'])
        
        recommendations = {}
        
        for product in self.inventory.keys():
            product_sales = df[df['product'] == product]
            
            if len(product_sales) > 0:
                # Calculate average daily sales
                daily_sales = product_sales.groupby(product_sales['date'].dt.date)['quantity'].sum()
                avg_daily_sales = daily_sales.mean()
                
                # Calculate safety stock (for 7 days)
                safety_stock = avg_daily_sales * 7
                
                # Recommend reorder level (safety stock + lead time demand)
                reorder_level = safety_stock + (avg_daily_sales * 3)  # 3 days lead time
                
                current_stock = self.inventory[product]['stock']
                
                recommendations[product] = {
                    'current_stock': current_stock,
                    'avg_daily_sales': round(avg_daily_sales, 2),
                    'recommended_reorder_level': round(reorder_level, 2),
                    'days_of_stock_remaining': round(current_stock / avg_daily_sales, 1) if avg_daily_sales > 0 else float('inf'),
                    'action': 'REORDER' if current_stock < reorder_level else 'OK'
                }
        
        return recommendations

class ReportGenerator:
    """Class for generating various reports"""
    
    def __init__(self, inventory, sales_history):
        self.inventory = inventory
        self.sales_history = sales_history
    
    def generate_inventory_report(self):
        """Generate comprehensive inventory report"""
        report = {
            'total_products': len(self.inventory),
            'total_value': sum(item['price'] * item['stock'] for item in self.inventory.values()),
            'low_stock_items': [],
            'high_value_items': [],
            'category_breakdown': defaultdict(lambda: {'count': 0, 'value': 0})
        }
        
        # Analyze each product
        for product, details in self.inventory.items():
            stock_value = details['price'] * details['stock']
            category = details['category']
            
            # Check for low stock
            if details['stock'] < 10:
                report['low_stock_items'].append({
                    'product': product,
                    'stock': details['stock'],
                    'price': details['price']
                })
            
            # Check for high value items
            if stock_value > 1000:
                report['high_value_items'].append({
                    'product': product,
                    'value': stock_value,
                    'stock': details['stock']
                })
            
            # Category breakdown
            report['category_breakdown'][category]['count'] += 1
            report['category_breakdown'][category]['value'] += stock_value
        
        return report
    
    def generate_financial_report(self, start_date=None, end_date=None):
        """Generate financial report for given period"""
        if not self.sales_history:
            return {}
        
        df = pd.DataFrame(self.sales_history)
        df['date'] = pd.to_datetime(df['date'])
        
        # Filter by date range if provided
        if start_date:
            df = df[df['date'] >= start_date]
        if end_date:
            df = df[df['date'] <= end_date]
        
        if len(df) == 0:
            return {}
        
        report = {
            'total_revenue': df['total'].sum(),
            'total_transactions': len(df),
            'total_items_sold': df['quantity'].sum(),
            'average_transaction_value': df['total'].mean(),
            'best_selling_products': df.groupby('product')['quantity'].sum().sort_values(ascending=False).head(5).to_dict(),
            'revenue_by_category': {},
            'daily_revenue': df.groupby(df['date'].dt.date)['total'].sum().to_dict()
        }
        
        # Revenue by category
        for product, total in df.groupby('product')['total'].sum().items():
            if product in self.inventory:
                category = self.inventory[product]['category']
                if category not in report['revenue_by_category']:
                    report['revenue_by_category'][category] = 0
                report['revenue_by_category'][category] += total
        
        return report

# Main execution
if __name__ == "__main__":
    try:
        # Create and run the grocery shop system
        system = GroceryShopSystem()
        system.run()
        
    except Exception as e:
        print(f"Error starting the application: {e}")
        import traceback
        traceback.print_exc()